In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np


# Load and process some slow signal data

In [ ]:
from ssm.core import pchain
from ssm.core.util_pmodules import Aggregate
from ssm.pmodules import *

# initializing our process chain
data_proc = pchain.ProcessingChain()

reader = Reader('/home/sflis/CTA/projects/SSM-analysis/data/astri_onsky/d2019-05-08/Run13312.hdf5')
data_proc.add(reader)
#This module removes incomplete frames and marks bad and unstable pixels
frame_cleaner = PFCleaner()
data_proc.add(frame_cleaner)
# A simple flat field computation based on the first 7000 frames
sff = SimpleFF(0,7000)
data_proc.add(sff)

#The Aggregate module collects the computed object from the frame
aggr = Aggregate(["raw_resp","simple_ff"])
data_proc.add(aggr)

# Simple visualization of the chain
print(data_proc)

#Execute the chain
data_proc.run()


# Plot the processed data

In [ ]:
from CHECLabPy.plotting.camera import CameraImage

data = aggr.aggr["raw_resp"][0]
ff = aggr.aggr['simple_ff'][0]

time_averaged = np.mean(data.data,axis=0)
camera_averaged = np.nanmean(data.data-ff,axis=1)
#A time series plot
plt.figure()
plt.plot(data.time-data.time[0],camera_averaged)

plt.xlabel('Time since run start (s)')
plt.ylabel("Average amplitude (mV)")

#Different average camera images
camera = CameraImage(data.xpix, data.ypix, data.size)
camera.image = time_averaged
camera.add_colorbar('Amplitdue (mV)')
camera.ax.set_title('Raw Data')

camera = CameraImage(data.xpix, data.ypix, data.size)
camera.image = time_averaged -ff
camera.add_colorbar('Amplitdue (mV)')
camera.ax.set_title('Flat fielded')

camera = CameraImage(data.xpix, data.ypix, data.size)
camera.image = ff
camera.add_colorbar('Amplitdue (mV)')
camera.ax.set_title('Flat field coefficents')
